In [11]:
import sqlite3
import csv
import pandas as pd
import os
import re

import math
from tqdm import tqdm
import multiprocessing as mp
import time
import sched
import ast

folder_prefix = '/home/jovyan/work/'

In [2]:
#update current_df_3 to include overlap == 0 and 1 of parsed_df_4's rows.

new_data = pd.read_csv(folder_prefix + "nowdata/parsing/current_df3_WEBTEXT.csv", sep="\t", low_memory=False, encoding="utf-8")

In [3]:
new_data

,CMO_NAME,CMO_MEMSUM,SCH_NAME,CMO_STATE,CMO_SCHNUM,CMO_URL,CMO_NUMSTATES,CMO_ALLSTATES,CMO_SECTOR,CMO_NUMSTUDENTS_CREDO17,...,YEAR_OPENED,YEAR_CLOSED,SEARCH16,TOTETH,PCTETH,AGE,PCTFRL,PLACE,STR,OVERLAPS_REMOVED
0,AAEC,1320.0,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,2010.0,NaN,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELL...,239.0,0.497917,5.0,-0.002083,Suburb,-480.000000,0
1,Arizona Agribusiness & Equine Center,1320.0,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,2010.0,NaN,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELL...,239.0,0.497917,5.0,-0.002083,Suburb,-480.000000,0
2,AAEC,1320.0,AAEC - SMCC CAMPUS,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,2005.0,NaN,"AAEC - SMCC CAMPUS 7050 S. 24th Street , Phoe...",396.0,0.912442,10.0,0.649770,City,-434.000000,0
3,Arizona Agribusiness & Equine Center,1320.0,AAEC - SMCC CAMPUS,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,2005.0,NaN,"AAEC - SMCC CAMPUS 7050 S. 24th Street , Phoe...",396.0,0.912442,10.0,0.649770,City,-434.000000,0
4,AAEC,1320.0,AAEC - PARADISE VALLEY,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,1998.0,NaN,AAEC - PARADISE VALLEY 17811 North 32nd Stree...,81.0,0.199507,17.0,-0.002463,City,-406.000000,0
5,Arizona Agribusiness & Equine Center,1320.0,AAEC - PARADISE VALLEY,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,1998.0,NaN,AAEC - PARADISE VALLEY 17811 North 32nd Stree...,81.0,0.199507,17.0,-0.002463,City,-406.000000,0
6,ACADEMY OF SKILLS AND KNOWLEDGE,1161.0,CUMBERLAND ACADEMY MIDDLE,TX,4.0,http://www.cumberlandacademy.com/,1.0,TX,NaN,NaN,...,2013.0,NaN,"CUMBERLAND ACADEMY MIDDLE 1040 SHILOH RD , TY...",240.0,0.471513,2.0,0.379175,City,12.469378,1
7,ACADEMY OF SKILLS AND KNOWLEDGE,1161.0,CUMBERLAND H S,TX,4.0,http://www.cumberlandacademy.com/,1.0,TX,NaN,NaN,...,2015.0,NaN,"CUMBERLAND H S 7200 PALUXY RD , TYLER TX 7570...",59.0,0.385621,0.0,NaN,Rural,NaN,1
8,ACADEMY OF SKILLS AND KNOWLEDGE,1161.0,CUMBERLAND ACADEMY,TX,4.0,http://www.cumberlandacademy.com/,1.0,TX,NaN,NaN,...,1998.0,NaN,"CUMBERLAND ACADEMY 1340 SHILOH RD , TYLER TX ...",217.0,0.434870,17.0,0.408818,City,13.773116,1
9,ACCELERATED INTERMEDIATE ACADEMY,281.0,ACCELERATED INTERDISCIPLINARY ACAD,TX,3.0,http://www.aiacharterschools.org/,1.0,TX,NaN,NaN,...,2004.0,NaN,ACCELERATED INTERDISCIPLINARY ACAD 12825 SUMM...,276.0,0.982206,11.0,0.864769,City,16.676558,1


In [4]:
#whatever's been parsed in df 4
parsed_df_4 =  pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_4.csv", sep="\t", low_memory=False, encoding="utf-8") 


In [9]:
parsed_df_4['NCESSCH']

0      4.002480e+10
1      4.000700e+10
2      4.800016e+11
3      4.000710e+10
4      4.000720e+10
5      4.000720e+10
6      4.000750e+10
7      4.000770e+10
8      6.227101e+10
9      6.227101e+10
10     4.800022e+11
11     4.800022e+11
12     4.800022e+11
13     4.800022e+11
14     4.800022e+11
15     4.800022e+11
16     4.800022e+11
17     4.800022e+11
18     4.800022e+11
19     1.100035e+11
20     2.601013e+11
21     3.600049e+11
22     3.600943e+11
23     3.600955e+11
24     3.600995e+11
25     3.601049e+11
26     3.601069e+11
27     3.601091e+11
28     4.800259e+11
29     6.227101e+10
           ...     
128    6.425101e+10
129    3.601047e+11
130    3.601072e+11
131    3.601088e+11
132    3.601136e+11
133    3.601041e+11
134    3.601073e+11
135    3.601103e+11
136    6.280501e+10
137    2.600312e+11
138    1.100095e+11
139    1.800071e+11
140    2.900579e+11
141    4.800016e+11
142    4.800022e+11
143    4.800016e+11
144    4.800022e+11
145    4.800016e+11
146    4.800022e+11


In [12]:
new_data['WEBTEXT'] = new_data['WEBTEXT'].fillna('0').apply(ast.literal_eval)

In [13]:
parsed_df_4['WEBTEXT'] = parsed_df_4['WEBTEXT'].fillna('0').apply(ast.literal_eval)

In [14]:
parsed_df_4['WEBTEXT'] [0]

[('http://www.telesis-academy.org/',
  'False',
  '0',
  "\t\t\t2018-05-24T11:56:26+00:00\t\t\t\t\t\t\t\t\t\nTelesis News\n2018 Graduation Tonight at 7pm (downloadable program)\nMay 23rd, 2018\nThe 2018 Telesis Graduation Ceremony is tonight at 7pm! Don't forget that RiverScene Magazine created a special SnapChat\n[...]\n2018-2019 School Calendar\nMay 7th, 2018\n[...]\nRedforEd and Thursday, April 26, 2018\nApril 23rd, 2018\n\xa0\r\r\xa0\rApril 23, 2018\rTelesis Preparatory Academy Parents and Guardians,\rThere has been a great deal of med\n[...]\nUpcoming Events\nQuick Links\nWhat's to eat?\nView Breakfast and Lunch Menus\r\nTelesis Food Services\nLinks & Information\r\nTelesis Enrollment Application\nThis is a printable version of our Enrollment Application. Please consider using our new online application \nhere\n.\r\nRenaissance Place\nAIA Academy BrainBook Login\nConcussion Awareness Education for Telesis Athletes\r\nStudent/Parent\nHandbook\n2018-2019 Student/Parent Handbook\nKno

In [17]:

for i in range(parsed_df_4.shape[0]):
    nc = parsed_df_4['NCESSCH'][i]
    i_d = new_data[new_data['NCESSCH'] == nc].index[0] #get index in 11K new_data with that index of given NCESSCH 
    #index_in_11K = new_data.get_loc[i_d] 
    new_data.loc[i_d, 'OVERLAPS_REMOVED'] = 1

In [24]:
for i in range(parsed_df_4.shape[0]):
    nc = parsed_df_4['NCESSCH'][i]
    i_d = new_data[new_data['NCESSCH'] == nc].index[0] #get index in 11K new_data with that index of given NCESSCH 
    #index_in_11K = new_data.get_loc[i_d] 
    if (new_data.loc[i_d, 'OVERLAPS_REMOVED'] != 1):
        print("not 1 at index : " + str(i_d))
    

In [20]:
parsed_df_3 =  pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_3.csv", sep="\t", low_memory=False, encoding="utf-8") 


In [21]:
for i in range(parsed_df_3.shape[0]):
    nc = parsed_df_3['NCESSCH'][i]
    i_d = new_data[new_data['NCESSCH'] == nc].index[0] #get index in 11K new_data with that index of given NCESSCH 
    #index_in_11K = new_data.get_loc[i_d] 
    if (new_data.loc[i_d, 'OVERLAPS_REMOVED'] != 1):
        print("not 1 at index : " + str(i_d))
    

In [22]:
parsed_df_2 =  pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_2.csv", sep="\t", low_memory=False, encoding="utf-8") 


In [23]:
for i in range(parsed_df_2.shape[0]):
    nc = parsed_df_2['NCESSCH'][i]
    i_d = new_data[new_data['NCESSCH'] == nc].index[0] #get index in 11K new_data with that index of given NCESSCH 
    #index_in_11K = new_data.get_loc[i_d] 
    if (new_data.loc[i_d, 'OVERLAPS_REMOVED'] != 1):
        print("not 1 at index : " + str(i_d))
    

In [28]:
new_data.to_csv(folder_prefix + "nowdata/parsing/current_df4_WEBTEXT.csv", mode="w", index=False, header=new_data.columns.values, sep="\t", encoding="utf-8")

In [29]:
look_new = pd.read_csv(folder_prefix + "nowdata/parsing/current_df4_WEBTEXT.csv", sep="\t", low_memory=False, encoding="utf-8")

In [30]:
look_new

,CMO_NAME,CMO_MEMSUM,SCH_NAME,CMO_STATE,CMO_SCHNUM,CMO_URL,CMO_NUMSTATES,CMO_ALLSTATES,CMO_SECTOR,CMO_NUMSTUDENTS_CREDO17,...,YEAR_OPENED,YEAR_CLOSED,SEARCH16,TOTETH,PCTETH,AGE,PCTFRL,PLACE,STR,OVERLAPS_REMOVED
0,AAEC,1320.0,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,2010.0,NaN,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELL...,239.0,0.497917,5.0,-0.002083,Suburb,-480.000000,0
1,Arizona Agribusiness & Equine Center,1320.0,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,2010.0,NaN,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELL...,239.0,0.497917,5.0,-0.002083,Suburb,-480.000000,0
2,AAEC,1320.0,AAEC - SMCC CAMPUS,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,2005.0,NaN,"AAEC - SMCC CAMPUS 7050 S. 24th Street , Phoe...",396.0,0.912442,10.0,0.649770,City,-434.000000,0
3,Arizona Agribusiness & Equine Center,1320.0,AAEC - SMCC CAMPUS,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,2005.0,NaN,"AAEC - SMCC CAMPUS 7050 S. 24th Street , Phoe...",396.0,0.912442,10.0,0.649770,City,-434.000000,0
4,AAEC,1320.0,AAEC - PARADISE VALLEY,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,1998.0,NaN,AAEC - PARADISE VALLEY 17811 North 32nd Stree...,81.0,0.199507,17.0,-0.002463,City,-406.000000,0
5,Arizona Agribusiness & Equine Center,1320.0,AAEC - PARADISE VALLEY,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,1998.0,NaN,AAEC - PARADISE VALLEY 17811 North 32nd Stree...,81.0,0.199507,17.0,-0.002463,City,-406.000000,0
6,ACADEMY OF SKILLS AND KNOWLEDGE,1161.0,CUMBERLAND ACADEMY MIDDLE,TX,4.0,http://www.cumberlandacademy.com/,1.0,TX,NaN,NaN,...,2013.0,NaN,"CUMBERLAND ACADEMY MIDDLE 1040 SHILOH RD , TY...",240.0,0.471513,2.0,0.379175,City,12.469378,1
7,ACADEMY OF SKILLS AND KNOWLEDGE,1161.0,CUMBERLAND H S,TX,4.0,http://www.cumberlandacademy.com/,1.0,TX,NaN,NaN,...,2015.0,NaN,"CUMBERLAND H S 7200 PALUXY RD , TYLER TX 7570...",59.0,0.385621,0.0,NaN,Rural,NaN,1
8,ACADEMY OF SKILLS AND KNOWLEDGE,1161.0,CUMBERLAND ACADEMY,TX,4.0,http://www.cumberlandacademy.com/,1.0,TX,NaN,NaN,...,1998.0,NaN,"CUMBERLAND ACADEMY 1340 SHILOH RD , TYLER TX ...",217.0,0.434870,17.0,0.408818,City,13.773116,1
9,ACCELERATED INTERMEDIATE ACADEMY,281.0,ACCELERATED INTERDISCIPLINARY ACAD,TX,3.0,http://www.aiacharterschools.org/,1.0,TX,NaN,NaN,...,2004.0,NaN,ACCELERATED INTERDISCIPLINARY ACAD 12825 SUMM...,276.0,0.982206,11.0,0.864769,City,16.676558,1


In [32]:
for i in range(parsed_df_4.shape[0]):
    nc = parsed_df_4['NCESSCH'][i]
    i_d = look_new[look_new['NCESSCH'] == nc].index[0] #get index in 11K new_data with that index of given NCESSCH 
    #index_in_11K = new_data.get_loc[i_d] 
    if (look_new.loc[i_d, 'OVERLAPS_REMOVED'] != 1):
        print("not 1 at index : " + str(i_d))